In [ ]:
from __future__ import division
from __future__ import print_function

from keras.models import model_from_json
import utils
import os
import cv2
import numpy as np
import pandas as pd
from scipy import ndimage 

In [ ]:
# Size of training image files.
image_size = (512,512)

# The test image path
TEST_PATH = "data/test/"

# The output path for visualizations.
OUTPUT_PATH = "output"

# The number of slices each image will be cut into.
n_slices = 32

# The ratio of the image.
ratio = (1, 1)

classes = ['vegetation', 'water', 'desert', 'clouds']

In [ ]:
# The path to save the model.
MODEL_PATH = "models"

# The name the model will be saved to.
model_name = "model"

In [ ]:
# load json and create model
json_file = open(os.path.join(MODEL_PATH, model_name + ".json"), 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights(os.path.join(MODEL_PATH, model_name + ".h5"))
print("Loaded model from disk")

In [ ]:
with open("test_images.txt", 'r') as f:
    image_urls = f.read().split("\n")

In [ ]:
for i, image_url in enumerate(image_urls):
    utils.download_image(image_url, os.path.join(TEST_PATH, '_'.join(image_url.split('/')[8:])))

In [ ]:
X_test = []
full_images = []
test_image_f_names = os.listdir(TEST_PATH)

for f_name in test_image_f_names:
    img = cv2.imread(os.path.join(TEST_PATH, f_name))    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    img_resize = cv2.resize(img, image_size)
    slices = utils.prepare_images(img_resize, image_size, ratio, n_slices)
    X_test.append(slices)
    full_images.append(img_resize)
    
full_images = np.stack(full_images)
X_test = np.stack(X_test)
X_test = X_test.reshape(X_test.shape[0] * X_test.shape[1], X_test.shape[2], X_test.shape[3], X_test.shape[4])

In [ ]:
preds = model.predict(X_test, verbose=1)

In [ ]:
labels = np.argmax(preds, axis=1)
labels = np.reshape(labels, (len(full_images), n_slices, n_slices))

In [ ]:
colors = {
    0:(124,252,0),
    1:(244,164,96),
    2:(0,191,255),
    3:(255, 255, 255)
}

In [ ]:
percentages = []

for i, (f_name, img, l) in enumerate(zip(test_image_f_names, full_images, labels)):
    assignment_mask = np.zeros((l.shape[0], l.shape[1], 3))
    percentages.append(np.bincount(l.flatten()) / l.size)
    
    for label in xrange(len(classes)):
        assignment_mask[np.isin(l, label)] = colors[label]
        
    alpha = 0.4
    output = img.copy()
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    output = output.astype('uint8')
    assignment_mask = assignment_mask.astype('uint8')
    assignment_mask = cv2.resize(assignment_mask, image_size, interpolation=cv2.INTER_CUBIC)
    assignment_mask = cv2.flip(ndimage.rotate(assignment_mask, 90),0)
    overlay = assignment_mask
    output = cv2.addWeighted(overlay, alpha, output, 1 - alpha,
                    0, output)
    cv2.imwrite(os.path.join(OUTPUT_PATH, f_name), img)
    cv2.imwrite(os.path.join(OUTPUT_PATH, "overlay_" + f_name), output)
    cv2.imwrite(os.path.join(OUTPUT_PATH, "mask_" + f_name), assignment_mask)

In [ ]:
results = pd.DataFrame(percentages, columns=classes)
results['f_name'] = test_image_f_names
results = results.fillna(0)

In [ ]:
results